In [1]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import ipynbname
import datetime
import sys
import random
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer

from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)


from Process_Function import RareCategoryTransformer
from Visualization_function import *
from LG_Aimers_6th.cal_auc import calculate_auc
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Load

In [2]:
data_seed = 7

train_path = f'../data/custom_train_{data_seed}.csv'
test_path = f'../data/custom_test_{data_seed}.csv'

# 학습/평가 데이터 로드
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

print(train.shape, test.shape)

(205080, 68) (51271, 67)


In [3]:
def drop_columns(df):
    cols = [
        '불임 원인 - 여성 요인',  # 고유값 1
        '불임 원인 - 정자 면역학적 요인',  # train, test 모두 '1'인 데이터 1개 >> 신뢰할 수 없음
        '난자 해동 경과일',
    ]
    df = df.drop(cols, axis=1)
    return df

def 특정시술유형(train, test):
    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" / ", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(":", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" ", "")

    counts = train['특정 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"
    test.loc[~test['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"

    train['특정 시술 유형'] = train['특정 시술 유형'].apply(categorize_procedure)
    test['특정 시술 유형'] = test['특정 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['특정 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['특정 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '특정 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 시술횟수(df_train):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상':'6회'})
        df_train[col] = df_train[col].str[0].astype(int)
    df_train['시술_임신'] = df_train['총 임신 횟수'] - df_train['총 시술 횟수']
    df_train = df_train.drop('총 시술 횟수', axis=1)
    return df_train

def 배란유도유형(df_train, df_test):
    mapping = {
        '기록되지 않은 시행': 1,
        '알 수 없음': 0,
        '세트로타이드 (억제제)': 0,
        '생식선 자극 호르몬': 0,
    }
    df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
    df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
    return df_train, df_test

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test

def 배아생성주요이유(df_train, df_test):
    df_train['배아 생성 주요 이유'] = df_train['배아 생성 주요 이유'].fillna('DI')
    df_test['배아 생성 주요 이유'] = df_test['배아 생성 주요 이유'].fillna('DI')

    df_train['배아 생성 이유 리스트'] = df_train['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])
    df_test['배아 생성 이유 리스트'] = df_test['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])

    mlb = MultiLabelBinarizer()
    train_one_hot = pd.DataFrame(
        mlb.fit_transform(df_train['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_train.index
    )
    train_one_hot.columns = ['배아생성이유_' + col for col in train_one_hot.columns]

    test_one_hot = pd.DataFrame(
        mlb.transform(df_test['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_test.index
    )
    test_one_hot.columns = ['배아생성이유_' + col for col in test_one_hot.columns]

    df_train = pd.concat([df_train, train_one_hot], axis=1)
    df_test = pd.concat([df_test, test_one_hot], axis=1)

    cols_to_drop = [
        '배아 생성 주요 이유',
        '배아 생성 이유 리스트',
        '배아생성이유_연구용',
        '배아생성이유_DI'
    ]
    df_train = df_train.drop(cols_to_drop, axis=1, errors='ignore')
    df_test = df_test.drop(cols_to_drop, axis=1, errors='ignore')

    cols = ['배아생성이유_기증용',
            '배아생성이유_난자 저장용',
            '배아생성이유_배아 저장용',
            '배아생성이유_현재 시술용']

    df_train[cols] = df_train[cols].div(df_train[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    df_test[cols] = df_test[cols].div(df_test[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

    return df_train, df_test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 기증자정자와혼합된난자수(df_train, df_test):
    df_train["기증자 정자와 혼합된 난자 수"] = df_train["기증자 정자와 혼합된 난자 수"].fillna(2)
    df_test["기증자 정자와 혼합된 난자 수"] = df_test["기증자 정자와 혼합된 난자 수"].fillna(2)
    return df_train, df_test

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cols_to_impute = [
        '임신 시도 또는 마지막 임신 경과 연수', # DI, IVF랑 관련 X
    ]
    imputer = SimpleImputer(strategy='mean')
    train[cols_to_impute] = imputer.fit_transform(train[cols_to_impute])
    test[cols_to_impute] = imputer.transform(test[cols_to_impute])

    cols_to_impute = [
        '난자 채취 경과일',
        '난자 혼합 경과일',
        '배아 이식 경과일',
        '배아 해동 경과일',

        '착상 전 유전 검사 사용 여부',
        'PGD 시술 여부',
        'PGS 시술 여부',

        ### DI only
        '착상 전 유전 진단 사용 여부',
        '총 생성 배아 수',
        '미세주입된 난자 수',
        '미세주입에서 생성된 배아 수',
        '이식된 배아 수',
        '미세주입 배아 이식 수',
        '저장된 배아 수',
        '미세주입 후 저장된 배아 수',
        '해동된 배아 수',
        '해동 난자 수',
        '수집된 신선 난자 수',
        '저장된 신선 난자 수',
        '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수',
        '기증자 정자와 혼합된 난자 수',
        '동결 배아 사용 여부',
        '신선 배아 사용 여부',
        '기증 배아 사용 여부',
        '대리모 여부',
        ### DI
    ]
    train[cols_to_impute] = train[cols_to_impute].fillna(0)
    test[cols_to_impute] = test[cols_to_impute].fillna(0)

    return train, test

def num_feature_scailing(train, test, seed=777):
    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '임신 성공 여부'
    ]

    arr_train = train[cols_to_scale].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[cols_to_scale].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[cols_to_scale]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    # train[cols_to_scale] = preprocessing.transform(arr_train + noise)
    train[cols_to_scale] = preprocessing.transform(arr_train)
    test[cols_to_scale] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

def all_process(train, val):
    # 기본 전처리 단계
    train, val = drop_columns(train), drop_columns(val)
    train, val = 특정시술유형(train, val)
    train, val = 시술횟수(train), 시술횟수(val)

    train, val = 단일배아이식여부(train, val)
    train, val = 배란유도유형(train, val)
    train, val = 배아생성주요이유(train, val)

    cols_to_encoding = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',
    ]
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


In [4]:
train

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부,시술유형_통합,시술_임신,배아생성이유_기증용,배아생성이유_난자 저장용,배아생성이유_배아 저장용,배아생성이유_현재 시술용
0,6,0,0.012546,0.292676,1,1.204935,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.955471,-0.000527,0,4,0.797886,-0.019561,-0.015913,-0.048814,0.109365
1,4,3,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.103057,-0.000527,0,6,-0.999986,-0.019561,-0.015913,-0.048814,0.109365
2,2,1,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.103057,-0.000527,0,6,0.797886,-0.019561,-0.015913,-0.048814,0.109365
3,5,0,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,-1.109261,-0.000527,0,6,0.797886,2.261183,-0.015913,-0.048814,-1.399908
4,2,0,0.012546,0.292676,1,1.204935,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.955471,-0.000527,1,6,-0.101796,-0.019561,-0.015913,-0.048814,0.109365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205075,2,3,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.103057,-0.000527,0,6,-1.346386,-0.019561,-0.015913,-0.048814,0.109365
205076,3,0,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.103057,-0.000527,0,5,-0.101796,-0.019561,-0.015913,-0.048814,0.109365
205077,0,1,0.012546,0.292676,1,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,0.955471,-0.000527,1,5,0.797886,-0.019561,-0.015913,-0.048814,0.109365
205078,3,0,0.012546,-1.200646,0,-0.288368,-0.012186,-0.01585,-0.033983,-0.014177,...,0.00014,-1.109261,-0.000527,0,0,-1.716415,-0.019561,-0.015913,-0.048814,-1.793899


In [5]:
trainer_config = TrainerConfig(
    batch_size=2048,
    # accumulate_grad_batches=1024,
    data_aware_init_batch_size=2000,
    auto_lr_find=False,
    max_epochs=100,
    accelerator='gpu',
    early_stopping='valid_loss',
    early_stopping_min_delta=0.001,
    early_stopping_mode='min',
    early_stopping_patience=3,
    checkpoints='valid_loss',
    checkpoints_path='saved_models',
    checkpoints_save_top_k=1,
    progress_bar='simple',
)
optimizer_config = OptimizerConfig()

In [6]:
n_splits = 5
seed_list = [333]
all_auc = []
test_preds = []
is_first = True

for seed in seed_list:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    auc_scores = []
    fold_test_preds = []

    train = pd.read_csv(train_path).drop(columns=['ID'])
    test = pd.read_csv(test_path).drop(columns=['ID'])

    for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['임신 성공 여부'])):
        fold_train, fold_valid = train.iloc[train_idx].copy().reset_index(drop=True), train.iloc[valid_idx].copy().reset_index(drop=True)
        fold_train2 = fold_train.copy()
        fold_test = test.copy()

        fold_train, fold_valid = all_process(fold_train, fold_valid)
        fold_train2, fold_test = all_process(fold_train2, fold_test)

        cat_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        numeric_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 여부']
        data_config = DataConfig(
            target=[
                "임신 성공 여부"
            ],
            continuous_cols=numeric_cols,
            categorical_cols=cat_cols,
        )

        model_config = GANDALFConfig(
            task="classification",
            gflu_stages=6,
            gflu_feature_init_sparsity=0.3,
            gflu_dropout=0.0,
            learning_rate=1e-3,
            metrics=["auroc"],
            metrics_prob_input=[True],
            seed=seed,
        )

        tabular_model = TabularModel(
            data_config=data_config,
            model_config=model_config,
            optimizer_config=optimizer_config,
            trainer_config=trainer_config,
            verbose=False,
        )

        tabular_model.fit(train=fold_train, validation=fold_valid, seed=seed)

        valid_preds = tabular_model.predict(fold_valid).iloc[:, 1]
        fold_auc = roc_auc_score(fold_valid['임신 성공 여부'], valid_preds)
        print(f'[Fold {fold + 1}] AUC: {fold_auc:.5f}')

        auc_scores.append(fold_auc)
        test_pred = tabular_model.predict(fold_test).iloc[:, 1]
        fold_test_preds.append(test_pred)

    test_preds.append(np.mean(fold_test_preds, axis=0))

    # 각 seed별 평균 AUC와 표준편차 출력
    seed_auc_mean = np.mean(auc_scores)
    seed_auc_std = np.std(auc_scores)
    all_auc.append(seed_auc_mean)
    print(f"Seed {seed} - Average AUC: {seed_auc_mean:.5f} (STD: {seed_auc_std:.5f})")
    print('=' * 60)



# 전체 결과에 대한 평균 및 표준편차 출력
total_auc_mean = np.mean(all_auc)
total_auc_std = np.std(all_auc)
print('-' * 60)
print(f'Total Average AUC: {total_auc_mean:.6f} (STD: {total_auc_std:.6f})')

Seed set to 333
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.0010964781961431851
Restoring states from the checkpoint path at C:\Users\eunhak\workspace\LG_Aimers_6th\Eunhak\.lr_find_309a2c90-a69d-413b-ba73-963c68c4e565.ckpt
Restored all states from the checkpoint at C:\Users\eunhak\workspace\LG_Aimers_6th\Eunhak\.lr_find_309a2c90-a69d-413b-ba73-963c68c4e565.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | _backbone        | GANDALFBackbone  | 256 K  | train
1 | _embedding_layer | Embedding1dLayer | 307    | train
2 | _head            | Sequential       | 172    | train
3 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
256 K     Trainable params
0         Non-trainable params
256 K     Total params
1.026     Total estimated model params size (MB)
35        Modules in train mode
0        

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[Fold 5] AUC: 0.73458
Seed 333 - Average AUC: 0.73401 (STD: 0.00144)
------------------------------------------------------------
Total Average AUC: 0.734012 (STD: 0.000000)


In [7]:
old_auc = 0.744533 * 100
old_std = 0.001171 * 100

new_auc = total_auc_mean * 100
new_std = total_auc_std * 100

def calculate_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value) * 100 if old_value != 0 else float('inf')
    return change, percentage_change

def format_change(change):
    return f"{change:+.6f}"

# 각 지표의 변화량 계산
auc_change, auc_pct = calculate_change(old_auc, new_auc)
std_change, std_pct = calculate_change(old_std, new_std)

# 결과 출력
print("\n========== 모델 성능 변화 ==========")
print(f"{'Metric':<8}  {'AUC':>12}  {'Acc':>12}")
print("-" * 36)
print(f"{'Old':<8}  {old_auc:>12.6f}  {old_std:>12.6f}")
print(f"{'New':<8}  {new_auc:>12.6f}  {new_std:>12.6f}")
print(f"{'Change':<8}  {format_change(auc_change):>12}  {format_change(std_change):>12}")
print(f"{'% Change':<8}  {auc_pct:>11.4f}%  {std_pct:>11.4f}%")
print("=" * 36)


========== 모델 성능 변화 ==========
Metric             AUC           Acc
------------------------------------
Old          74.453300      0.117100
New          73.401160      0.000000
Change       -1.052140     -0.117100
% Change      -1.4132%    -100.0000%


In [8]:
tmp_submission = pd.DataFrame({f'gandalf_{data_seed}': np.mean(test_preds, axis=0)})
tmp_submission

,gandalf_7
0,0.380250
1,0.232263
2,0.369434
3,0.010583
4,0.000806
...,...
51266,0.472226
51267,0.414249
51268,0.001705
51269,0.572522


In [9]:
from LG_Aimers_6th.cal_auc import calculate_auc

score = calculate_auc(tmp_submission, seed=data_seed)
print(f'[seed {data_seed}]: {score}')

[seed 7]: 0.7396985888734923
